In [1]:
#%matplotlib ipympl
from typing import Literal
from pathlib import Path
from matplotlib import pyplot as plt
#from mpl_interactions import ipyplot as iplt
import numpy as np
#from ipywidgets import IntSlider
from tqdm import tqdm
import pandas as pd

In [2]:
input = """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
"""
foyer_size = (11, 7)

In [70]:
input = Path("input.txt").read_text()
foyer_size = (101, 103)
# has period of 10578

In [4]:
V2 = tuple[int,int]
def add(a:V2, b:V2)->V2:
    return (a[0]+b[0], a[1]+b[1])
def scale(a:V2, s:int)->V2:
    return (a[0]*s, a[1]*s)
def mod(a:V2, m:V2)->V2:
    return (a[0]%m[0], a[1]%m[1])
def quad(a:V2, foyer:V2)->Literal[0,1,2,3,4]:
    hx = foyer[0]//2
    hy = foyer[1]//2
    if a[0]==hx or a[1]==hy:
        return 0
    match (a[0]<hx, a[1]<hy):
        case True, True:
            return 1
        case False, True:
            return 2
        case True, False:
            return 3
        case False, False:
            return 4
def move(bot:tuple[V2, V2], steps:int, foyer_size:V2):
    return mod(add(bot[0], scale(bot[1], steps)), foyer_size)

In [ ]:
bots:list[tuple[V2, V2]] = [
    tuple(
        tuple(int(a) for a in ass[2:].split(","))
        for ass in 
        item.split(" ")
    )
    for item in input.strip().splitlines()
]

In [ ]:
def generate(steps:int):
    fbots = [
        move(bot, steps, foyer_size)
        for bot
        in bots
    ]
    block = []
    for y in range(foyer_size[1]):
        row = []
        for x in range(foyer_size[0]):
            count = 0
            for fbot in fbots:
                if fbot[0]==x and fbot[1]==y:
                    count+=1
            row.append(count)
        block.append(row)
    return np.array(block)

In [ ]:
def plot(state):
    plt.imshow(state)
    plt.show()

In [ ]:
# My first approach would have found it just fine; It was just too slow
for steps in tqdm(range(6300,6400)):
    state = generate(steps)
    view = np.lib.stride_tricks.sliding_window_view(state>0, (3,3))
    has33 = view.all(axis=-1).all(axis=-1).any()
    if has33:
        print(steps)
        plot(state)

In [ ]:
# why did this work?
res_acc = []
for steps in range(0, 10578):
    fbots = np.array([
        move(bot, steps, foyer_size)
        for bot
        in bots
    ])
    res_acc.append(np.abs(np.sort(fbots[:,0])-np.sort(fbots[:,1])).sum()) # day 1 solution as metric
scores = pd.Series(res_acc)
# Used this to figure out the period:
# scores.iloc[0:10578].loc[res>8000].plot()

res_acc = np.zeros(foyer_size)
for steps, score in tqdm(zip(range(0, 10578), scores.values)):
    if score>5000:
        for (x,y) in [
            move(bot, steps, foyer_size)
            for bot
            in bots
        ]:
            res_acc[x][y] += 1
plt.imshow(res_acc)

In [ ]:
# here we search the suspect area for high robot counts
res_acc = []
for steps in range(0, 10578):
    fbots = np.array([
        move(bot, steps, foyer_size)
        for bot
        in bots
    ])
    x = fbots[:,0]
    y = fbots[:,1]
    res_acc.append(np.sum((x>60) & (x<80) & (y>60) & (y<80)))
scores_box = pd.Series(res_acc)
print(scores_box.argmax())
scores_box.plot()

In [ ]:
plt.imshow(generate(6355))
plt.show()